In [0]:
import numpy as np
import matplotlib.pyplot as plt

In [0]:
'''Compares two squares of (at the moment 9x9 size - adjustable if neural net is retrained)
and gives a 'likeliness' value for it, meaning that the ones with the highest likeliness
value can be selected to generate the disparity map'''
def compare(left_square, right_square): #here we define the compare function-other people are working on it
    # Here goes the Neural Network to do actual comparison, for now it will just return a random value.
    return np.random.rand()

In [0]:
'''Iterates through all the pixels of the image and calls the "find_matching_pixel" 
function for each one - also generates a depthmap based on the absolute depth at the same time'''
def find_all_pixels(left_image, right_image, square_size, nth_pixel_checked, sa_height, sa_width):
  depthmap = np.zeros((len(left_image), len(left_image[0])))
  for i in range(square_size, len(left_image)):
    for j in range(square_size, len(left_image[0]), nth_pixel_checked):
      x, y = find_matching_pixel(left_image, right_image, square_size, nth_pixel_checked, sa_height, sa_width, i, j)
      depthmap[i][j] = depth(np.sqrt((x-i)**2+(y-j)**2))
  return depthmap


In [0]:
'''Iterates through a preset search area and finds the best matching pixel so that
after calling this function for each pixel on the image a depthmap can be generated'''
def find_matching_pixel(left_image, right_image, square_size, nth_pixel_checked, sa_height, sa_width, x, y):
        #you go to left picture and pick an initial pixel and the square size are around it
        left_square = left_image[x-square_size:x+square_size+1, y-square_size:y+square_size+1] 
    
        #define search area in which I look for same pixels (that will be at the RIGHT image)
        #start of search area
        starting_x = max(square_size, x-sa_width) # make sure doesn't go outside of actual image 
        starting_y = max(square_size, y-sa_height)
        #end of search area
        ending_x = min(len(left_image)-square_size, x)
        ending_y = min(len(left_image[0])-square_size, y+sa_height+1)

        #now I need to see how much the two areas are alike (LIKELINESS), then figure out to which pixel on left image that corresponds (BEST MATCH) 
        highest_likeness = 0 #at the beginning, before comparing the picture, the likeliness is 0
        best_match_position = (0,0) #your initial best match is 0
        for i in range(starting_x, ending_x): #for every i and
            for j in range(starting_y, ending_y, nth_pixel_checked): #for every j in the search area
                right_square = right_image[i-square_size:i+square_size+1, j-square_size:j+square_size+1] #define right image and from which to which element it goes
        #by now I have itinerated through every part pixel (and its square_size surroundings) of the search area  
                #rint(right_square)
                likeness = compare(left_square, right_square) #this function will compare the likeness of each right square to the left square
                if likeness > highest_likeness: #if the new likeness is bigger than the set highest likeness (at the beginning, 0)
                    highest_likeness = likeness #set your new likeness as your highest likeliness
                    best_i, best_j = i, j #also, set your best match position (to which the new highest likeness corresponds) to their new values i, j
        #now you know where the pixel from the left image is located on the right image! :D        
        return best_i, best_j

In [0]:
''' computes absolute distance based on the disparity of the pixel '''
def depth(disparity):
    f=1 #focal length of the camera
    B=0.2 #distance between camera centers
    z=f*B/disparity #z is depth-you get distance in meters
    #scaling of z: what is the furthest away point? depends on in which environment we intend to use it.150?
    # max_distance=150
    # deep=255#scale it to get a scale 0-255
    return z

In [0]:
if __name__ == '__main__': #This is the main function that can then call the other functions
    image_height = 480
    image_width = 640
    square_size = 5 #we can modify later (nn will figure out what value fits best)
    nth_pixel_checked = 1 # the location of every nth (where n is the value of the variable) pixel will be checked (NN modifies later)
    sa_height, sa_width = 2, 20 # height of area searched, width of area searched (NN modifies later)
    # generate random "images" (matrices) - before we get real NN
    left_image = np.random.rand(480, 640, 3)
    right_image = np.random.rand(480, 640, 3)
    left_image = np.pad(left_image,[(square_size, square_size), (square_size, square_size), (0,0)], constant_values=-1)
    right_image = np.pad(right_image,[(square_size, square_size), (square_size, square_size), (0,0)], constant_values=-1)
    # just visualized the input images
    plt.imshow(left_image)
    plt.figure()
    plt.imshow(right_image)
    # calls find_pixels function and passes all the arguments
    depthmap = find_all_pixels(left_image, right_image, square_size, nth_pixel_checked, sa_height, sa_width) 
    plt.figure()
    plt.imshow(depthmap)